# Inspecting the Calculation Logic


This notebook demonstrates how to inspect intermediate calculation results by outputting cached properties of various IScope objects.

In this notebook, we use the Template example. The last line of the code below executes `template.py` and imports `workspace` into this notebook.

In [1]:
import pandas as pd
from template import workspace

`workspace` is an instance of the IfrsWorkspace class. The `models` property of `workspace` holds a dictionary that contains `IModel` objects. These objects are keyed by `ImportArgs` objects, which represent the associated reporting node, reporting period, and import format.

In [2]:
list(workspace.models.keys())

[ImportArgs(ReportingNode='CH', Year=2020, Month=12, Periodicity=0, Scenario='', ImportFormat='Opening'),
 ImportArgs(ReportingNode='CH', Year=2020, Month=12, Periodicity=0, Scenario='', ImportFormat='Cashflow'),
 ImportArgs(ReportingNode='CH', Year=2020, Month=12, Periodicity=0, Scenario='', ImportFormat='Actual'),
 ImportArgs(ReportingNode='CH', Year=2021, Month=3, Periodicity=0, Scenario='', ImportFormat='Cashflow'),
 ImportArgs(ReportingNode='CH', Year=2021, Month=3, Periodicity=0, Scenario='', ImportFormat='Actual')]

## Inspecting PresentValue IScope

The code below assigns the second-to-last model in the dictionary to `model_cf` for later use.

In [3]:
model_cf = list(workspace.models.values())[-2]

This model holds cached IScope objects that are created after importing the nominal cashflow data for the period ending March 2021. The cached IScopes are stored in a nested dictionary assigned to the `scope_cache` property of `model_cf`.

At the top level, `scope_cache` is keyed by IScope classes. The following code lists the classes of the cached IScopes.

In [4]:
list(model_cf.scope_cache.keys())

[ifrs17.ImportScopeCalculation.GetCashflowIdentities,
 ifrs17.ImportScopeCalculation.ComputeIfrsVarsCashflows,
 ifrs17.ImportScopeCalculation.PvLocked,
 ifrs17.ImportScopeCalculation.ValidAmountType,
 ifrs17.ImportScopeCalculation.EmptyValuesAocStep,
 ifrs17.ImportScopeCalculation.PvCurrent,
 ifrs17.ImportScopeCalculation.RaCurrent,
 ifrs17.ImportScopeCalculation.RaLocked,
 ifrs17.ImportScopeCalculation.CurrentPeriodAmortizationFactor,
 ifrs17.ImportScopeCalculation.MonthlyAmortizationFactorCashflow,
 ifrs17.ImportScopeCalculation.NominalCashflow,
 ifrs17.ImportScopeCalculation.ReferenceAocStep,
 ifrs17.ImportScopeCalculation.CoverageUnitCashflow,
 ifrs17.ImportScopeCalculation.DiscountedCashflow,
 ifrs17.ImportScopeCalculation.MonthlyRate,
 ifrs17.ImportScopeCalculation.DefaultValueBeExperienceAdjustmentForPremium,
 ifrs17.ImportScopeCalculation.PresentValueFromDiscountedCashflow,
 ifrs17.ImportScopeCalculation.CashflowAocStep,
 ifrs17.ImportScopeCalculation.BeExperienceAdjustmentForP

For each class key, a nested dictionary is assigned. The keys of this nested dictionary represent the `context` of the stored IScope objects. For most IScope objects, `context` signifies the `EconomicBasis` key. However, for certain IScope objects, such as `AllocateTechnicalMargin`, `context` corresponds to `EstimateTypes`. The `valid_context` attribute defined in each IScope class indicates what context values are acceptable for the class.  
If an IScope does not have a context, the `''` (empty string) is assigned.

The code below reveals that two context keys, `L` and `C`, are found for the `PresentValue` IScope.

In [5]:
from ifrs17.ImportScopeCalculation import PresentValue
list(model_cf.scope_cache[PresentValue].keys())

['L', 'C']

The innermost dictionaries in the nested `scope_cache` dictionary store IScope objects keyed by identity objects. The code blow lists the first 5 keys for the `PresentValue` IScope with context `L`.

In [6]:
list(model_cf.scope_cache[PresentValue]['L'].keys())[:5]

[IdentityTuple(Id=ImportIdentity(DataNode='DT1.1', AocType='CL', Novelty='C', IsReinsurance=False, ValuationApproach='BBA', ProjectionPeriod=0, ImportScope=None), AmountType='PR', EstimateType='BE', AccidentYear=0.0),
 IdentityTuple(Id=ImportIdentity(DataNode='DT1.1', AocType='CL', Novelty='C', IsReinsurance=False, ValuationApproach='BBA', ProjectionPeriod=0, ImportScope=None), AmountType='NIC', EstimateType='BE', AccidentYear=0.0),
 IdentityTuple(Id=ImportIdentity(DataNode='DT1.1', AocType='CL', Novelty='C', IsReinsurance=False, ValuationApproach='BBA', ProjectionPeriod=0, ImportScope=None), AmountType='ICO', EstimateType='BE', AccidentYear=0.0),
 IdentityTuple(Id=ImportIdentity(DataNode='DT1.1', AocType='CL', Novelty='C', IsReinsurance=False, ValuationApproach='BBA', ProjectionPeriod=0, ImportScope=None), AmountType='', EstimateType='RA', AccidentYear=0.0),
 IdentityTuple(Id=ImportIdentity(DataNode='DT1.1', AocType='EV', Novelty='N', IsReinsurance=False, ValuationApproach='BBA', Proj

Types of the identity objects vary by IScopes. In the case of `PresentValue`, `IdentityTuple` is used. Other types, such as `IdentityTuple2` or `IdentityTuple3` are used for some other IScopes.

To view the cached values of an IScope's properties, you can use the `debug` method, which outputs the values as a DataFrame. The first parameter is the name of the IScope to examine. The `debug` method can also accept additional parameters to filter the results. These parameters can be object identities, such as `AocType`, `Novelty`, and `AmountType`. Additionally, `context` can be used to filter the results.

The code below outputs `PresentValue` IScopes for DataNode='DT1.1', Novelty='N', EconomicBasis='L' and EstimateType='BE'. Note the parameter name should be `context` to indicate `EconomicBasis`, not `EconomicBasis` itself.

In [7]:
model_cf.debug('PresentValue', DataNode='DT1.1', Novelty='N', context='L', EstimateType='BE')

Scope DataNode AocType Novelty AmountType  \
0   PresentValueFromDiscountedCashflow    DT1.1     BOP       N         PR   
1   PresentValueFromDiscountedCashflow    DT1.1     BOP       N        NIC   
2   PresentValueFromDiscountedCashflow    DT1.1     BOP       N        ICO   
3                      CashflowAocStep    DT1.1      CF       N         PR   
4                      CashflowAocStep    DT1.1      CF       N        NIC   
5                      CashflowAocStep    DT1.1      CF       N        ICO   
6                         PresentValue    DT1.1      EV       N         PR   
7                         PresentValue    DT1.1      EV       N        NIC   
8                         PresentValue    DT1.1      EV       N        ICO   
9    PresentValueWithInterestAccretion    DT1.1      IA       N         PR   
10   PresentValueWithInterestAccretion    DT1.1      IA       N        NIC   
11   PresentValueWithInterestAccretion    DT1.1      IA       N        ICO   

   EstimateType  AccidentYear  \
0            BE           0.0   
1            BE           0.0   
2            BE           0.0   
3            BE           0.0   
4            BE           0.0   
5            BE           0.0   
6            BE           0.0   
7            BE           0.0   
8            BE           0.0   
9            BE           0.0   
10           BE           0.0   
11           BE           0.0   

                                               Values EconomicBasis  \
0   [-39.963392949885524, -39.97004741764894, -29....             L   
1   [47.94808795107775, 43.95607198301388, 39.9633...             L   
2   [11.987021987769438, 10.98901799575347, 9.9908...             L   
3   [0.0, 10.0, 0.0, 0.0, 10.0, 0.0, 0.0, 10.0, 0....             L   
4   [-4.0, -4.0, -4.0, -4.0, -4.0, -4.0, -4.0, -4....             L   
5   [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....             L   
6   [19.981696474942762, 19.98502370882447, 14.987...             L   
7   [59.93510993884718, 54.945089978767335, 49.954...             L   
8   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...             L   
9   [-0.006654467763418193, -0.0049904349878137876...             L   
10  [0.007984031936129716, 0.0073193050546098174, ...             L   
11  [0.001996007984032429, 0.0018298262636524544, ...             L   

        Value  shift  timeStep  \
0  -39.963393    0.0       3.0   
1   47.948088    0.0       3.0   
2   11.987022    0.0       3.0   
3   10.000000    0.0       3.0   
4  -12.000000    0.0       3.0   
5   -3.000000    0.0       3.0   
6   14.990015    0.0       3.0   
7   44.962557    0.0       3.0   
8    0.000000    0.0       3.0   
9   -0.016636    0.0       3.0   
10   0.021958    0.0       3.0   
11   0.005489    0.0       3.0   

                               parentDiscountedValues  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9   [39.963392949885524, 39.97004741764894, 29.975...   
10  [-47.94808795107775, -43.95607198301388, -39.9...   
11  [-11.987021987769438, -10.98901799575347, -9.9...   

                                  parentNominalValues  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                                 NaN   
7                      

By default, `debug` includes the results of sub classes of the specified IScope class. To exclude them, set the `include_sub` paramter to `False`.

In [8]:
model_cf.debug('PresentValue', DataNode='DT1.1', Novelty='N', context='L', EstimateType='BE', include_sub=False)

Scope DataNode AocType Novelty AmountType EstimateType  \
0  PresentValue    DT1.1      EV       N         PR           BE   
1  PresentValue    DT1.1      EV       N        NIC           BE   
2  PresentValue    DT1.1      EV       N        ICO           BE   

   AccidentYear EconomicBasis  \
0           0.0             L   
1           0.0             L   
2           0.0             L   

                                              Values      Value  shift  \
0  [19.981696474942762, 19.98502370882447, 14.987...  14.990015      0   
1  [59.93510993884718, 54.945089978767335, 49.954...  44.962557      0   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   0.000000      0   

   timeStep  
0         3  
1         3  
2         3

## Inspecting AllocateTechnical Margin

In the same way as we did above for the `PresentValue` IScope, let's take a look at `AllocateTechnicalMargin`.
The code below extracts the last model in the `models` proeprty of `workspace`.

In [9]:
model_act = list(workspace.models.values())[-1]

The code below lists IScope types calculated after importing the actual cashflow.

In [10]:
list(model_act.scope_cache.keys())

[ifrs17.ImportScopeCalculation.GetActualIdentities,
 ifrs17.ImportScopeCalculation.AllCfIdentities,
 ifrs17.ImportScopeCalculation.ComputeIfrsVarsActuals,
 ifrs17.ImportScopeCalculation.Actual,
 ifrs17.ImportScopeCalculation.BeAmountTypesFromIfrsVariables,
 ifrs17.ImportScopeCalculation.EmptyValuesActual,
 ifrs17.ImportScopeCalculation.AdvanceActual,
 ifrs17.ImportScopeCalculation.OverdueActual,
 ifrs17.ImportScopeCalculation.DefaultValueActualExperienceAdjustmentOnPremium,
 ifrs17.ImportScopeCalculation.AmortizationDeferrable,
 ifrs17.ImportScopeCalculation.PreviousAocSteps,
 ifrs17.ImportScopeCalculation.DeferrableActual,
 ifrs17.ImportScopeCalculation.ReleaseDeferrable,
 ifrs17.ImportScopeCalculation.ActualBase,
 ifrs17.ImportScopeCalculation.AmfFromIfrsVariable,
 ifrs17.ImportScopeCalculation.ContractualServiceMargin,
 ifrs17.ImportScopeCalculation.AllocateTechnicalMargin,
 ifrs17.ImportScopeCalculation.TechnicalMarginForAM,
 ifrs17.ImportScopeCalculation.TechnicalMarginForBOP,
 if

The code below outputs AllocateTechnicalMargin values for CSM of DataNode='DT1.1'. Note the paramter `context` now indicates `EstimateType` to be selected. `'C'` indicates Contractural Service Margin, where `L` and `LR` indicates Loss Component and Loss Recovery Component respectively. 

In [18]:
model_act.debug('AllocateTechnicalMargin', DataNode='DT1.1', context='C') # context='L' selects Loss Component.

Scope DataNode AocType Novelty  \
0         AllocateTechnicalMargin    DT1.1      AM       C   
1         AllocateTechnicalMargin    DT1.1      AU       I   
2         AllocateTechnicalMargin    DT1.1      AU       N   
3         AllocateTechnicalMargin    DT1.1      CF       C   
4         AllocateTechnicalMargin    DT1.1      CF       I   
5         AllocateTechnicalMargin    DT1.1      CF       N   
6         AllocateTechnicalMargin    DT1.1      EV       I   
7         AllocateTechnicalMargin    DT1.1      EV       N   
8         AllocateTechnicalMargin    DT1.1     CRU       I   
9         AllocateTechnicalMargin    DT1.1      EA       C   
10        AllocateTechnicalMargin    DT1.1      MC       I   
11        AllocateTechnicalMargin    DT1.1     RCU       I   
12        AllocateTechnicalMargin    DT1.1      IA       I   
13        AllocateTechnicalMargin    DT1.1     YCU       I   
14        AllocateTechnicalMargin    DT1.1      IA       N   
15  AllocateTechnicalMarginForBop    DT1.1     BOP       I   
16  AllocateTechnicalMarginForBop    DT1.1     BOP       N   
17   AllocateTechnicalMarginForCl    DT1.1      CL       C   
18  AllocateTechnicalMarginForEop    DT1.1     EOP       C   

    AggregatedTechnicalMargin  TechnicalMargin ComputedEstimateType HasSwitch  \
0                  -19.503146         6.289625                    C     False   
1                 -148.934502         0.000000                    C     False   
2                   31.975040         0.000000                    L     False   
3                    0.000000         0.000000                    C     False   
4                 -148.860127         0.000000                    C     False   
5                   31.958739         0.000000                    L     False   
6                 -148.934502         0.000000                    C     False   
7                   31.975040        55.456316                    L     False   
8                 -148.934502         0.000000                    C     False   
9                  -61.503146        42.000000                    C     False   
10                 -42.968114      -105.892014                    C     False   
11                -148.860127         0.000000                    C     False   
12                -148.860127        -0.074374                    C     False   
13                -148.934502         0.000000                    C     False   
14                  31.958739         0.016301                    L     False   
15                   0.000000       -42.968114                    C     False   
16                   0.000000        31.958739                    L     False   
17                  87.431356         0.000000                    C      True   
18                 -13.213521       -52.513713                    C     False   

   EstimateType       Value  balancingValue  
0             C    6.289625             NaN  
1             C    0.000000             NaN  
2             C    0.000000             NaN  
3             C    0.000000             NaN  
4             C    0.000000             NaN  
5             C    0.000000             NaN  
6             C    0.000000             NaN  
7             C    0.000000             NaN  
8             C    0.000000             NaN  
9             C   42.000000             NaN  
10            C -105.892014             NaN  
11            C    0.000000             NaN  
12            C   -0.074374             NaN  
13            C    0.000000             NaN  
14            C    0.000000             NaN  
15            C  -42.968114             NaN  
16            C    0.000000             NaN  
17            C   87.431356       87.431356  
18            C  -13.213521             NaN